In [90]:
import ast
from pathlib import Path
from typing import List, Dict, Set
from typeguard import typechecked
from itertools import zip_longest
from pyvis.network import Network
from numpy import log as ln

@typechecked
def imports_from_file(file: Path) -> List[str]:
    ''' Use ast to extract all imports from file
    '''
    class ImportVisitor(ast.NodeVisitor):

        def __init__(self):
            self.imports = set()

        def visit_Import(self, import_node):
            for alias in import_node.names:
                self.imports.add(alias.name)
            super(ImportVisitor, self).generic_visit(import_node)

        def visit_ImportFrom(self, import_from_node):
            m1 = import_from_node.module
            if (m1 is not None):
                for m2 in import_from_node.names:
                    self.imports.add(m1 + '.' + m2.name)
            super(ImportVisitor, self).generic_visit(import_from_node)

    try:
        file_ast = ast.parse(open(file).read())
    except:
        print(f"Error parsing {file}")
        return []
    visitor = ImportVisitor()        
    visitor.visit(file_ast)
    imports = list(visitor.imports)
    imports.sort()
    return imports

@typechecked
def is_module_in_package(module: str, package: str) -> bool:
    ''' Check if module (e.g. 'x.y.z') is part of a package (e.g. 'x.y')
    '''
    for m, p in zip_longest(module.split('.'), package.split('.')):
        if (m != p and p != None):
            return False
    return True

assert(is_module_in_package('zeeguu_core.util.hash', 'zeeguu_core.util'))
assert(not is_module_in_package('zeeguu_core.util.hash', 'zeeguu_core.x'))
assert(not is_module_in_package('zeeguu_core.util', 'zeeguu_core.util.hash'))

@typechecked
def import_info_from_file(file: Path, info: Dict[str, int]) -> None:
    ''' Examine all imports from file and update the info dict
        
        info dict is a map from package to import count and only imports
        to packages already in the info dict is considered
    '''
    for i in imports_from_file(file):
        for p in info:
            if (is_module_in_package(i, p)):
                info[p] = info[p] + 1

@typechecked
def LOC(file: Path) -> int:
    ''' Return the number of Lines Of Code in a file
        TODO: discount comment lines?
    '''
    return sum([1 for line in open(file)])


@typechecked
def module_view_from_packages(packages: List[Path], source_folder: Path, network_size: str = '500px') -> [Network, Dict[str, Dict]]:
    ''' TODO
    '''
    # The dict of infos for each package - we will add to the inner dict below
    package_infos: Dict[str, Dict] =  { str(p.relative_to(source_folder)).replace('\\','.'): {'path': p} for p in packages}

    # Add list of files to inner package_infos dict
    for name, info in package_infos.items():
        files = [file for file in info['path'].rglob('*.py')]
        info['files'] = files
    
    # Add import info to inner dict in package_infos
    for name, info in package_infos.items():
        import_info = {k:0 for k in package_infos.keys()}
        for file in info['files']:
            import_info_from_file(file, import_info)
        info['import_info'] = import_info
       
    # Add total LOC to inner dict in package_infos
    total_loc = 0
    for name, info in package_infos.items():
        loc = sum([LOC(f) for f in info['files']])
        info['LOC'] = loc
        total_loc += loc

    # Create network
    net = Network(directed=True, notebook=True, height=network_size, width=network_size)
    net.set_edge_smooth('dynamic')

    # Nodes
    size_scale = 100/(len(package_infos)**0.1*total_loc)
    for name, info in package_infos.items():
        net.add_node(name, size = size_scale*info['LOC'], title = name + " LOC: " + str(info['LOC']), label=name)

    # Edges
    for name, info in package_infos.items():
        for dependency, count in info['import_info'].items():
            if (name != dependency and count > 0):
                net.add_edge(name, dependency, title = str(count), width = ln(count), arrowStrikethrough=False)

    return [net, package_infos]

In [91]:
# Root folder of all source code analyzed
source_folder: Path = Path('C:/Users/Carsten/source/repos/Zeeguu-API')

# Generate module view using top level folders
packages: List[Path] = [x for x in source_folder.iterdir() if x.is_dir() and not x.name.startswith('.')]
[net, package_infos] = module_view_from_packages(packages, source_folder)
net.show('module_view_simple.html')


In [92]:
# Find the file in zeeguu_core that has a reference to zeeguu_api
p = package_infos['zeeguu_core']
for f in p['files']:
    imports = imports_from_file(f)
    for i in imports:
        if i.startswith('zeeguu_api'):
            print(f)

C:\Users\Carsten\source\repos\Zeeguu-API\zeeguu_core\emailer\zeeguu_mailer.py


In [93]:
# Generate module view using top level folders and sub-folders of zeeguu_core
packages: List[Path] = [x for x in source_folder.iterdir() if x.is_dir() and not x.name.startswith('.') and not x.name.startswith('zeeguu_core')]
packages.extend([x for x in (source_folder/'zeeguu_core').iterdir() if x.is_dir() and not x.name.startswith('.')])

[net, package_infos] = module_view_from_packages(packages, source_folder, '1000px')
net.show('module_view_detailed.html')


Class User
[]
